###### Imports and Settings

In [1]:
import pandas as pd
import numpy as np
import requests
import pickle
from collections import deque
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

# American Community Survey 2006-2010 5 Year Estimates

In [2]:
#read in API key
with open('api_keys.pkl', 'rb') as keys_file:
        keys_dict_2 = pickle.load(keys_file)

In [3]:
#variable containing Census API key
api_key = keys_dict_2['CENSUS']

In [4]:
GNRC = ['111', #Macon
       '161', #Stewart
       '125', #Montgomery
       '083', #Houston
       '085', #Humphreys
       '043', #Dickson
       '021', #Cheatham
       '147', #Robertson
       '165', #Sumner
       '037', #Davidson
       '189', #Wilson
       '169', #Trousdale
       '187', #Williamson
       '149', #Rutherford
       '119'] #Maury
KY = ['003', #Allen
      '213'] #Simpson
qlaces = ['1600000US4700200', #Adams city, Tennessee: Robertson
          '1600000US4702180', #Ashland City town, Tennessee: Cheatham
          '1600000US4704620', #Belle Meade city, Tennessee: Davidson
          '1600000US4705140', #Berry Hill city, Tennessee: Davidson
          '1600000US4708280', #Brentwood city, Tennessee: Williamson
          '1600000US4709880', #Burns town, Tennessee: Dickson
          '1600000US4711980', #Cedar Hill city, Tennessee: Robertson
          '1600000US4713080', #Charlotte town, Tennessee: Dickson
          '1600000US4715160', #Clarksville city, Tennessee: Montgomery
          '1600000US4716540', #Columbia city, Tennessee: Maury
          '1600000US4716980', #Coopertown town, Tennessee: Robertson
          '1600000US4718420', #Cross Plains city, Tennessee: Robertson
          '1600000US4718820', #Cumberland City town, Tennessee: Stewart
          '1600000US4720620', #Dickson city, Tennessee: Dickson
          '1600000US4721400', #Dover city, Tennessee: Stewart
          '1600000US4722360', #Eagleville city, Tennessee: Rutherford
          '1600000US4724320', #Erin city, Tennessee: Houston
          '1600000US4725440', #Fairview city, Tennessee: Williamson
          '1600000US4727020', #Forest Hills city, Tennessee: Davidson
          '1600000US4727740', #Franklin city, Tennessee: Williamson
          '1600000US4728540', #Gallatin city, Tennessee: Sumner
          '1600000US4729920', #Goodlettsville city, Tennessee: Davidson/Sumner
          '1600000US4730960', #Greenbrier town, Tennessee: Robertson
          '1600000US4733280', #Hendersonville city, Tennessee: Sumner
          '1600000US4739660', #Kingston Springs town, Tennessee: Cheatham
          '1600000US4741200', #La Vergne city, Tennessee: Rutherford
          '1600000US4740160', #La Vergne city, Tennessee: Macon
          '1600000US4741520', #Lebanon city, Tennessee: Wilson
          '1600000US4744840', #McEwen city, Tennessee: Humphreys
          '1600000US4748980', #Millersville city, Tennessee: Robertson/Sumner
          '1600000US4749460', #Mitchellville city, Tennessee: Sumner
          '1600000US4750780', #Mount Juliet city, Tennessee: Wilson
          '1600000US4751080', #Mount Pleasant city, Tennessee: Maury
          '1600000US4751560', #Murfreesboro city, Tennessee: Rutherford
          '1600000US4752006', #Nashville-Davidson metropolitan government (balance): Davidson
          '1600000US4752820', #New Johnsonville city, Tennessee: Humphreys
          '1600000US4753460', #Nolensville town, Tennessee: Williamson
          '1600000US4754780', #Oak Hill city, Tennessee: Davidson
          '1600000US4757480', #Pegram town, Tennessee: Cheatham
          '1600000US4759560', #Pleasant View city, Tennessee: Cheatham
          '1600000US4760280', #Portland city, Tennessee: Robertson/Sumner
          '1600000US4763140', #Ridgetop city, Tennessee: Davidson/Robertson
          '1600000US4769080', #Slayden town, Tennessee: Dickson
          '1600000US4769420', #Smyrna town, Tennessee: Rutherford
          '1600000US4770580', #Spring Hill city, Tennessee: Maury/Williamson
          '1600000US4770500', #Springfield city, Tennessee: Robertson
          '1600000US4773460', #Tennessee Ridge town, Tennessee: Houston/Stewart
          '1600000US4773900', #Thompson's Station town, Tennessee: Williamson
          '1600000US4776860', #Vanleer town, Tennessee: Dickson
          '1600000US4778320', #Watertown city, Tennessee: Wilson
          '1600000US4778560', #Waverly city, Tennessee: Humphreys
          '1600000US4779420', #Westmoreland town, Tennessee: Sumner
          '1600000US4779980', #White Bluff town, Tennessee: Dickson
          '1600000US4780200', #White House city, Tennessee: Robertson/Sumner
          '1600000US2128918', #Franklin city, Kentucky: 
          '1600000US2169114'] #Scottsville city, Kentucky

## Read In Data Guide

In [5]:
dataguide = pd.read_csv('../../Data Guides/DATA GUIDE ACS 2020 5YR.csv', dtype = str)
dataguide['ID'] = dataguide['ID'].astype(int)

In [6]:
dg1 = dataguide[dataguide['ID'].between(1, 46)]
dg2 = dataguide[dataguide['ID'].between(47, 92)]
dg3 = dataguide[dataguide['ID'].between(93, 138)]
dg4 = dataguide[dataguide['ID'].between(139, 184)]
dg5 = dataguide[dataguide['ID'].between(185, 230)]
dg6 = dataguide[dataguide['ID'].between(231, 276)]
dg7 = dataguide[dataguide['ID'].between(277, 322)]
dg8 = dataguide[dataguide['ID'].between(323, 368)]
dg9 = dataguide[dataguide['ID'].between(369, 414)]
dg10 = dataguide[dataguide['ID'].between(415, 460)]
dg11 = dataguide[dataguide['ID'].between(461, 506)]
dg12 = dataguide[dataguide['ID'].between(507, 552)]
dg13 = dataguide[dataguide['ID'].between(553, 598)]
dg14 = dataguide[dataguide['ID'].between(599, 644)]
dg15 = dataguide[dataguide['ID'].between(645, 690)]
dg16 = dataguide[dataguide['ID'].between(691, 736)]
dg17 = dataguide[dataguide['ID'].between(737, 782)]
dg18 = dataguide[dataguide['ID'].between(783, 828)]
dg19 = dataguide[dataguide['ID'].between(829, 874)]
dg20 = dataguide[dataguide['ID'].between(875, 920)]
dg21 = dataguide[dataguide['ID'].between(921, 966)]
dg22 = dataguide[dataguide['ID'].between(967, 1012)]
dg23 = dataguide[dataguide['ID'].between(1013, 1058)]
dg24 = dataguide[dataguide['ID'].between(1059, 1104)]
dg25 = dataguide[dataguide['ID'].between(1105, 1150)]
dg26 = dataguide[dataguide['ID'].between(1151, 1196)]
dg27 = dataguide[dataguide['ID'].between(1197, 1242)]
dg28 = dataguide[dataguide['ID'].between(1243, 1287)]
dg29 = dataguide[dataguide['ID'].between(1288, 1332)]
dg30 = dataguide[dataguide['ID'].between(1333, 1377)]
dg31 = dataguide[dataguide['ID'].between(1378, 1422)]
dg32 = dataguide[dataguide['ID'].between(1423, 1467)]
dg33 = dataguide[dataguide['ID'].between(1468, 1512)]
dg34 = dataguide[dataguide['ID'].between(1513, 1557)]
dg35 = dataguide[dataguide['ID'].between(1558, 1602)]
dg36 = dataguide[dataguide['ID'].between(1603, 1647)]
dg37 = dataguide[dataguide['ID'].between(1648, 1692)]
dg38 = dataguide[dataguide['ID'].between(1693, 1737)]
dg39 = dataguide[dataguide['ID'].between(1738, 1782)]
dg40 = dataguide[dataguide['ID'].between(1783, 1827)]
dg41 = dataguide[dataguide['ID'].between(1828, 1872)]
dg42 = dataguide[dataguide['ID'].between(1873, 1917)]
dg43 = dataguide[dataguide['ID'].between(1918, 1962)]
dg44 = dataguide[dataguide['ID'].between(1963, 2007)]
dg45 = dataguide[dataguide['ID'].between(2008, 2052)]
dg46 = dataguide[dataguide['ID'].between(2053, 2097)]
dg47 = dataguide[dataguide['ID'].between(2098, 2142)]
dg48 = dataguide[dataguide['ID'].between(2143, 2187)]
dg49 = dataguide[dataguide['ID'].between(2188, 2232)]
dg50 = dataguide[dataguide['ID'].between(2233, 2277)]
dg51 = dataguide[dataguide['ID'].between(2278, 2322)]
dg52 = dataguide[dataguide['ID'].between(2323, 2367)]
dg53 = dataguide[dataguide['ID'].between(2368, 2412)]

## One Through Thirty Seven

In [7]:
#url string and list parameters for column head and tail
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
head1 = 'NAME' 
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'

In [8]:
dataguide = dg1
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [9]:
one = savename
one.head()

,NAME,GEO_ID,agebysex_total_series,age_total_male,age_m_u5,age_m_5to9,age_m_10to14,age_m_15to17,age_m_18to19,age_m_20,age_m_21,age_m_22to24,age_m_25to29,age_m_30to34,age_m_35to39,age_m_40to44,age_m_45to49,age_m_50to54,age_m_55to59,age_m_60to61,age_m_62to64,age_m_65to66,age_m_67to69,age_m_70to74,age_m_75to79,age_m_80to84,age_m_85+,age_total_female,age_f_u5,age_f_5to9,age_f_10to14,age_f_15to17,age_f_18to19,age_f_20,age_f_21,age_f_22to24,age_f_25to29,age_f_30to34,age_f_35to39,age_f_40to44,age_f_45to49,age_f_50to54,age_f_55to59,age_f_60to61,age_f_62to64,age_f_65to66,age_f_67to69,age_f_70to74,StateFIPS,GeoFIPS
1,"Montgomery County, Tennessee",0500000US47125,204992,102205,8916,7959,7229,4053,2544,1642,1694,5927,10985,9174,7666,6094,5506,5259,4623,2116,2378,1684,1912,1930,1315,866,733,102787,8202,7512,7228,3840,2668,2397,1482,4333,10075,8753,7521,6207,5940,5600,5312,2537,2555,2069,1672,2656,47,125
9,"Rutherford County, Tennessee",0500000US47149,324139,159299,10772,11045,12034,6813,4930,3181,3526,8239,12814,11257,11656,10646,10800,9812,9546,3665,3593,2739,3316,3684,2688,1316,1227,164840,10320,10622,11492,6779,4741,3030,3124,8844,12414,11585,11896,11522,11063,10321,9809,3692,4493,2658,3906,5067,47,149
15,"Sumner County, Tennessee",0500000US47165,187680,91606,5776,6523,6312,3938,2181,872,1088,3242,5890,5639,6489,5687,6458,6340,5896,2673,3321,2179,2878,3393,2521,1235,1075,96074,5516,6640,5672,3817,1955,927,916,3131,5945,6258,6047,6680,6682,6517,6800,2473,3490,2429,3302,3882,47,165
16,"Trousdale County, Tennessee",0500000US47169,10910,6668,373,425,268,160,349,115,191,281,926,837,374,474,342,310,412,89,119,143,113,165,155,23,24,4242,142,301,219,105,72,0,31,87,373,425,151,340,295,303,279,155,133,161,151,182,47,169
22,"Williamson County, Tennessee",0500000US47187,232380,113906,6932,9230,9690,6180,3047,1379,799,3825,4969,5484,7735,8706,9206,8230,7606,3159,3736,2259,3016,3778,1950,1919,1071,118474,6765,8827,9418,5973,2535,1313,1121,3547,5088,6306,8009,9437,9481,8690,8376,3177,3827,2562,2925,4457,47,187


In [10]:
dataguide = dg2
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [11]:
two = savename

In [12]:
dataguide = dg3
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [13]:
three = savename

In [14]:
dataguide = dg4
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [15]:
four = savename

In [16]:
dataguide = dg5
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [17]:
five = savename

In [18]:
dataguide = dg6
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [19]:
six = savename

In [20]:
dataguide = dg7
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [21]:
seven = savename

In [22]:
dataguide = dg8
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [23]:
eight = savename

In [24]:
dataguide = dg9
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [25]:
nine = savename

In [26]:
dataguide = dg10
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [27]:
ten = savename

In [28]:
dataguide = dg11
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [29]:
eleven = savename

In [30]:
dataguide = dg12
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [31]:
twelve = savename

In [32]:
dataguide = dg13
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [33]:
thirteen = savename

In [34]:
dataguide = dg14
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [35]:
fourteen = savename

In [36]:
dataguide = dg15
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [37]:
fifteen = savename

In [38]:
dataguide = dg16
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [39]:
sixteen = savename

In [40]:
dataguide = dg17
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [41]:
seventeen = savename

In [42]:
dataguide = dg18
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [43]:
eighteen = savename

In [44]:
dataguide = dg19
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [45]:
nineteen = savename

In [46]:
dataguide = dg20
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [47]:
twenty = savename

In [48]:
dataguide = dg21
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [49]:
twentyone = savename

In [50]:
dataguide = dg22
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [51]:
twentytwo = savename

In [52]:
dataguide = dg23
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [53]:
twentythree = savename

In [54]:
dataguide = dg24
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [55]:
twentyfour = savename

In [56]:
dataguide = dg25
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [57]:
twentyfive = savename

In [58]:
dataguide = dg26
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [59]:
twentysix = savename

In [60]:
dataguide = dg27
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [61]:
twentyseven = savename

In [62]:
dataguide = dg28
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [63]:
twentyeight = savename

In [64]:
dataguide = dg29
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [65]:
twentynine = savename

In [66]:
dataguide = dg30
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [67]:
thirty = savename

In [68]:
dataguide = dg31
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [69]:
thirtyone = savename

In [70]:
dataguide = dg32
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [71]:
thirtytwo = savename

In [72]:
dataguide = dg33
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [73]:
thirtythree = savename

In [74]:
dataguide = dg34
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [75]:
thirtyfour = savename

In [76]:
dataguide = dg35
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [77]:
thirtyfive = savename

In [78]:
dataguide = dg36
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [79]:
thirtysix = savename

In [80]:
dataguide = dg37
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [81]:
thirtyseven = savename

In [82]:
dataguide = dg38
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [83]:
thirtyeight = savename

In [84]:
dataguide = dg39
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [85]:
thirtynine = savename

In [86]:
dataguide = dg40
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [87]:
forty = savename

In [88]:
dataguide = dg41
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [89]:
fortyone = savename

In [90]:
dataguide = dg42
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [91]:
fortytwo = savename

In [92]:
dataguide = dg43
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [93]:
fortythree = savename

In [94]:
dataguide = dg44
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [95]:
fortyfour = savename

In [96]:
dataguide = dg45
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [97]:
fortyfive = savename

In [98]:
dataguide = dg46
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [99]:
fortysix = savename

In [100]:
dataguide = dg47
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [101]:
fortyseven = savename

In [102]:
dataguide = dg48
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [103]:
fortyeight = savename

In [104]:
dataguide = dg49
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [105]:
fortynine = savename

In [106]:
dataguide = dg50
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [107]:
fifty = savename

In [108]:
dataguide = dg51
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [109]:
fiftyone = savename

In [110]:
dataguide = dg52
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [111]:
fiftytwo = savename

In [112]:
dataguide = dg53
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [113]:
fiftythree = savename

In [115]:
last = fiftythree

In [116]:
dfs = [two,three,four,five,six,seven,eight,nine,ten,eleven,twelve,thirteen, fourteen, fifteen, sixteen, seventeen, eighteen, nineteen, twenty, twentyone,
       twentytwo, twentythree, twentyfour, twentyfive, twentysix, twentyseven, twentyeight, twentynine, thirty, thirtyone, thirtytwo, thirtythree, 
       thirtyfour, thirtyfive, thirtysix, thirtyseven, thirtyeight, thirtynine, forty, fortyone, fortytwo, fortythree, fortyfour, fortyfive, 
       fortysix, fortyseven, fortyeight, fortynine, fifty, fiftyone, fiftytwo]

In [117]:
for df in dfs:
    df = df.drop(columns = ['NAME','StateFIPS','GeoFIPS'], inplace = True)

In [118]:
one = one.drop(columns = ['StateFIPS','GeoFIPS'])
last = last.drop(columns = 'NAME')

In [119]:
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['GEO_ID'],
                                            how='outer'), dfs)

In [120]:
dfs = [one, df_merged, last]
data = reduce(lambda  left,right: pd.merge(left,right,on=['GEO_ID'],
                                            how='outer'), dfs)

In [121]:
data = data.set_index(['NAME', 'GEO_ID'])
data.head()

agebysex_total_series age_total_male age_m_u5 age_m_5to9 age_m_10to14 age_m_15to17 age_m_18to19  \
NAME                         GEO_ID                                                                                                           
Montgomery County, Tennessee 0500000US47125                204992         102205     8916       7959         7229         4053         2544   
Rutherford County, Tennessee 0500000US47149                324139         159299    10772      11045        12034         6813         4930   
Sumner County, Tennessee     0500000US47165                187680          91606     5776       6523         6312         3938         2181   
Trousdale County, Tennessee  0500000US47169                 10910           6668      373        425          268          160          349   
Williamson County, Tennessee 0500000US47187                232380         113906     6932       9230         9690         6180         3047   

                                            age_m_20 age_m_21 age_m_22to24 age_m_25to29 age_m_30to34 age_m_35to39 age_m_40to44 age_m_45to49  \
NAME                         GEO_ID                                                                                                           
Montgomery County, Tennessee 0500000US47125     1642     1694         5927        10985         9174         7666         6094         5506   
Rutherford County, Tennessee 0500000US47149     3181     3526         8239        12814        11257        11656        10646        10800   
Sumner County, Tennessee     0500000US47165      872     1088         3242         5890         5639         6489         5687         6458   
Trousdale County, Tennessee  0500000US47169      115      191          281          926          837          374          474          342   
Williamson County, Tennessee 0500000US47187     1379      799         3825         4969         5484         7735         8706         9206   

                                            age_m_50to54 age_m_55to59 age_m_60to61 age_m_62to64 age_m_65to66 age_m_67to69 age_m_70to74 age_m_75to79  \
NAME                         GEO_ID                                                                                                                   
Montgomery County, Tennessee 0500000US47125         5259         4623         2116         2378         1684         1912         1930         1315   
Rutherford County, Tennessee 0500000US47149         9812         9546         3665         3593         2739         3316         3684         2688   
Sumner County, Tennessee     0500000US47165         6340         5896         2673         3321         2179         2878         3393         2521   
Trousdale County, Tennessee  0500000US47169          310          412           89          119          143          113          165          155   
Williamson County, Tennessee 0500000US47187         8230         7606         3159         3736         2259         3016         3778         1950   

                                            age_m_80to84 age_m_85+ age_total_female age_f_u5 age_f_5to9 age_f_10to14 age_f_15to17 age_f_18to19  \
NAME                         GEO_ID                                                                                                              
Montgomery County, Tennessee 0500000US47125          866       733           102787     8202       7512         7228         3840         2668   
Rutherford County, Tennessee 0500000US47149         1316      1227           164840    10320      10622        11492         6779         4741   
Sumner County, Tennessee     0500000US47165         1235      1075            96074     5516       6640         5672         3817         1955   
Trousdale County, Tennessee  0500000US47169           23        24             4242      142        301          219          105           72   
Williamson County, Tennessee 0500000US47187         1919      1071           118474     6765       8827   

In [122]:
transp = data.transpose()
transp.head()

NAME,"Montgomery County, Tennessee","Rutherford County, Tennessee","Sumner County, Tennessee","Trousdale County, Tennessee","Williamson County, Tennessee","Wilson County, Tennessee","Cheatham County, Tennessee","Macon County, Tennessee","Robertson County, Tennessee","Stewart County, Tennessee","Davidson County, Tennessee","Dickson County, Tennessee","Houston County, Tennessee","Humphreys County, Tennessee","Maury County, Tennessee","Simpson County, Kentucky","Allen County, Kentucky","Adams city, Tennessee","Ashland City town, Tennessee","Belle Meade city, Tennessee","Berry Hill city, Tennessee","Brentwood city, Tennessee","Burns town, Tennessee","Cedar Hill city, Tennessee","Charlotte town, Tennessee","Clarksville city, Tennessee","Columbia city, Tennessee","Coopertown town, Tennessee","Cross Plains city, Tennessee","Cumberland City town, Tennessee","Dickson city, Tennessee","Dover city, Tennessee","Eagleville city, Tennessee","Erin city, Tennessee","Fairview city, Tennessee","Forest Hills city, Tennessee","Franklin city, Tennessee","Gallatin city, Tennessee","Goodlettsville city, Tennessee","Greenbrier town, Tennessee","Hendersonville city, Tennessee","Kingston Springs town, Tennessee","Lafayette city, Tennessee","La Vergne city, Tennessee","Lebanon city, Tennessee","McEwen city, Tennessee","Millersville city, Tennessee","Mitchellville city, Tennessee","Mount Juliet city, Tennessee","Mount Pleasant city, Tennessee","Murfreesboro city, Tennessee","Nashville-Davidson metropolitan government (balance), Tennessee","New Johnsonville city, Tennessee","Nolensville town, Tennessee","Oak Hill city, Tennessee","Pegram town, Tennessee","Pleasant View city, Tennessee","Portland city, Tennessee","Ridgetop city, Tennessee","Slayden town, Tennessee","Smyrna town, Tennessee","Springfield city, Tennessee","Spring Hill city, Tennessee","Tennessee Ridge town, Tennessee","Thompson's Station town, Tennessee","Vanleer town, Tennessee","Watertown city, Tennessee","Waverly city, Tennessee","Westmoreland town, Tennessee","White Bluff town, Tennessee","White House city, Tennessee","Scottsville city, Kentucky","Franklin city, Kentucky",Tennessee,United States
GEO_ID,0500000US47125,0500000US47149,0500000US47165,0500000US47169,0500000US47187,0500000US47189,0500000US47021,0500000US47111,0500000US47147,0500000US47161,0500000US47037,0500000US47043,0500000US47083,0500000US47085,0500000US47119,0500000US21213,0500000US21003,1600000US4700200,1600000US4702180,1600000US4704620,1600000US4705140,1600000US4708280,1600000US4709880,1600000US4711980,1600000US4713080,1600000US4715160,1600000US4716540,1600000US4716980,1600000US4718420,1600000US4718820,1600000US4720620,1600000US4721400,1600000US4722360,1600000US4724320,1600000US4725440,1600000US4727020,1600000US4727740,1600000US4728540,1600000US4729920,1600000US4730960,1600000US4733280,1600000US4739660,1600000US4740160,1600000US4741200,1600000US4741520,1600000US4744840,1600000US4748980,1600000US4749460,1600000US4750780,1600000US4751080,1600000US4751560,1600000US4752006,1600000US4752820,1600000US4753460,1600000US4754780,1600000US4757480,1600000US4759560,1600000US4760280,1600000US4763140,1600000US4769080,1600000US4769420,1600000US4770500,1600000US4770580,1600000US4773460,1600000US4773900,1600000US4776860,1600000US4778320,1600000US4778560,1600000US4779420,1600000US4779980,1600000US4780200,1600000US2169114,1600000US2128918,0400000US47,0100000US
agebysex_total_series,204992,324139,187680,10910,232380,140604,40539,24208,70982,13553,690540,53289,8201,18528,94615,18319,21065,602,4720,2191,1220,42700,1711,271,2115,156092,39515,4528,1900,336,15500,1661,943,1706,8989,4868,80675,40262,16829,6859,57888,2744,5239,35556,34759,1802,6403,160,35834,4899,141704,667070,1930,9221,4536,2492,4660,12928,2441,123,50820,17147,41797,1296,6116,418,1698,4124,2622,3583,12258,4489,8909,6772268,326569308
age_total_male,102205,159299,91606,6668,113906,68930,20221,11894,35176,6572,332527,26246,4119,9113,45700,9099,10453,305,2298,1044,543,21154,839,138,113

In [123]:
numcols = list(transp.columns)
numcols
transp[numcols] = transp[numcols].astype(float)

In [124]:
data = transp

In [125]:
GNRCCounties = [data[('Stewart County, Tennessee', '0500000US47161')],data[('Montgomery County, Tennessee', '0500000US47125')],
                data[('Houston County, Tennessee', '0500000US47083')],data[('Humphreys County, Tennessee', '0500000US47085')],
                data[('Dickson County, Tennessee', '0500000US47043')],data[('Cheatham County, Tennessee', '0500000US47021')],
                data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
                data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
                data[('Trousdale County, Tennessee', '0500000US47169')],data[('Williamson County, Tennessee', '0500000US47187')],
                data[('Rutherford County, Tennessee', '0500000US47149')]]
data['GNRC'] = sum(GNRCCounties)
GNRCCountiesAll = [data[('Stewart County, Tennessee', '0500000US47161')],data[('Montgomery County, Tennessee', '0500000US47125')],
                   data[('Houston County, Tennessee', '0500000US47083')],data[('Humphreys County, Tennessee', '0500000US47085')],
                   data[('Dickson County, Tennessee', '0500000US47043')],data[('Cheatham County, Tennessee', '0500000US47021')],
                   data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
                   data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
                   data[('Trousdale County, Tennessee', '0500000US47169')],data[('Williamson County, Tennessee', '0500000US47187')],
                   data[('Rutherford County, Tennessee', '0500000US47149')],data[('Maury County, Tennessee', '0500000US47119')]]
data['GNRC Region'] = sum(GNRCCountiesAll)
MPOCounties = [data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
               data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
               data[('Williamson County, Tennessee', '0500000US47187')],data[('Rutherford County, Tennessee', '0500000US47149')],
               data[('Maury County, Tennessee', '0500000US47119')]]
data['MPO'] = sum(MPOCounties)
RuthInc = [data[('Eagleville city, Tennessee', '1600000US4722360')],data[('La Vergne city, Tennessee', '1600000US4741200')],
           data[('Murfreesboro city, Tennessee', '1600000US4751560')],data[('Smyrna town, Tennessee', '1600000US4769420')]]
data[('Rutherford Incorporated', 'None')] = sum(RuthInc)
data[('Rutherford Unincorporated', 'None')] = data[('Rutherford County, Tennessee', '0500000US47149')] - data[('Rutherford Incorporated', 'None')]
WilsonInc = [data[('Lebanon city, Tennessee', '1600000US4741520')],data[('Mount Juliet city, Tennessee', '1600000US4750780')],
             data[('Watertown city, Tennessee', '1600000US4778320')]]
data[('Wilson Incorporated', 'None')] = sum(WilsonInc)
data[('Wilson Unincorporated', 'None')] = data[('Wilson County, Tennessee', '0500000US47189')] - data[('Wilson Incorporated', 'None')]
CheathInc = [data[('Ashland City town, Tennessee', '1600000US4702180')],data[('Kingston Springs town, Tennessee', '1600000US4739660')],
             data[('Pegram town, Tennessee', '1600000US4757480')],data[('Pleasant View city, Tennessee', '1600000US4759560')]]
data[('Cheatham Incorporated', 'None')] = sum(CheathInc)
data[('Cheatham Unincorporated', 'None')] = data[('Cheatham County, Tennessee', '0500000US47021')] - data[('Cheatham Incorporated', 'None')]
DicksInc = [data[('Burns town, Tennessee', '1600000US4709880')],data[('Charlotte town, Tennessee', '1600000US4713080')],
            data[('Dickson city, Tennessee', '1600000US4720620')],data[('Slayden town, Tennessee', '1600000US4769080')],
            data[('Vanleer town, Tennessee', '1600000US4776860')],data[('White Bluff town, Tennessee', '1600000US4779980')]]
data[('Dickson Incorporated', 'None')] = sum(DicksInc)
data[('Dickson Unincorporated', 'None')] = data[('Dickson County, Tennessee', '0500000US47043')] - data[('Dickson Incorporated', 'None')]
HumphInc = [data[('McEwen city, Tennessee', '1600000US4744840')],data[('New Johnsonville city, Tennessee', '1600000US4752820')],
            data[('Waverly city, Tennessee', '1600000US4778560')]]
data[('Humphreys Incorporated', 'None')] = sum(HumphInc)
data[('Humphreys Unincorporated', 'None')] = data[('Humphreys County, Tennessee', '0500000US47085')] - data[('Humphreys Incorporated', 'None')]
data[('Montgomery Incorporated', 'None')] = data[('Clarksville city, Tennessee', '1600000US4715160')]
data[('Montgomery Unincorporated', 'None')] = data[('Montgomery County, Tennessee', '0500000US47125')] - data[('Montgomery Incorporated', 'None')]

In [126]:
ok = data.transpose().reset_index()

In [127]:
ok.head()

NAME          GEO_ID  agebysex_total_series  age_total_male  age_m_u5  age_m_5to9  age_m_10to14  age_m_15to17  \
0  Montgomery County, Tennessee  0500000US47125               204992.0        102205.0    8916.0      7959.0        7229.0        4053.0   
1  Rutherford County, Tennessee  0500000US47149               324139.0        159299.0   10772.0     11045.0       12034.0        6813.0   
2      Sumner County, Tennessee  0500000US47165               187680.0         91606.0    5776.0      6523.0        6312.0        3938.0   
3   Trousdale County, Tennessee  0500000US47169                10910.0          6668.0     373.0       425.0         268.0         160.0   
4  Williamson County, Tennessee  0500000US47187               232380.0        113906.0    6932.0      9230.0        9690.0        6180.0   

   age_m_18to19  age_m_20  age_m_21  age_m_22to24  age_m_25to29  age_m_30to34  age_m_35to39  age_m_40to44  age_m_45to49  age_m_50to54  age_m_55to59  \
0        2544.0    1642.0    1694.0        5927.0       10985.0        9174.0        7666.0        6094.0        5506.0        5259.0        4623.0   
1        4930.0    3181.0    3526.0        8239.0       12814.0       11257.0       11656.0       10646.0       10800.0        9812.0        9546.0   
2        2181.0     872.0    1088.0        3242.0        5890.0        5639.0        6489.0        5687.0        6458.0        6340.0        5896.0   
3         349.0     115.0     191.0         281.0         926.0         837.0         374.0         474.0         342.0         310.0         412.0   
4        3047.0    1379.0     799.0        3825.0        4969.0        5484.0        7735.0        8706.0        9206.0        8230.0        7606.0   

   age_m_60to61  age_m_62to64  age_m_65to66  age_m_67to69  age_m_70to74  age_m_75to79  age_m_80to84  age_m_85+  age_total_female  age_f_u5  \
0        2116.0        2378.0        1684.0        1912.0        1930.0        1315.0         866.0      733.0          102787.0    8202.0   
1        3665.0        3593.0        2739.0        3316.0        3684.0        2688.0        1316.0     1227.0          164840.0   10320.0   
2        2673.0        3321.0        2179.0        2878.0        3393.0        2521.0        1235.0     1075.0           96074.0    5516.0   
3          89.0         119.0         143.0         113.0         165.0         155.0          23.0       24.0            4242.0     142.0   
4        3159.0        3736.0        2259.0        3016.0        3778.0        1950.0        1919.0     1071.0          118474.0    6765.0   

   age_f_5to9  age_f_10to14  age_f_15to17  age_f_18to19  age_f_20  age_f_21  age_f_22to24  age_f_25to29  age_f_30to34  age_f_35to39  age_f_40to44  \
0      7512.0        7228.0        3840.0        2668.0    2397.0    1482.0        4333.0       10075.0        8753.0        7521.0        6207.0   
1     10622.0       11492.0        6779.0        4741.0    3030.0    3124.0        8844.0       12414.0       11585.0       11896.0       11522.0   
2      6640.0        5672.0        3817.0        1955.0     927.0     916.0        3131.0        5945.0        6258.0        6047.0        6680.0   
3       301.0         219.0         105.0          72.0       0.0      31.0          87.0         373.0         425.0         151.0         340.0   
4      8827.0        9418.0        5973.0        2535.0    1313.0    1121.0        3547.0        5088.0        6306.0        8009.0        9437.0   

   age_f_45to49  age_f_50to54  age_f_55to59  age_f_60to61  age_f_62to64  age_f_65to66  age_f_67to69  age_f_70to74  age_f_75to79  age_f_80to84  \
0        5940.0        5600.0        5312.0        2537.0        2555.0        2069.0        1672.0        2656.0        1754.0        1130.0   
1       11063.0       10321.0        9809.0        3692.0        4493.0        2658.0        3906.0        5067.0        3530.0        1952.0   
2        6682.0        6517.0        6800.0        2473.0        3490.0        2429.0        330

In [128]:
ok.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Columns: 2375 entries, NAME to GeoFIPS
dtypes: float64(2373), object(2)
memory usage: 1.6+ MB


In [129]:
ok.to_csv('../../Data/ACS20205YR.csv', index = False)